In [1]:
import pandas as pd # for dataframes
import numpy as np # for numerical computing and linear algebra data structures and algorithms
from numba import jit # for JIT compilation facilties for improved efficiency

In [2]:
train_beh = pd.read_csv('./Data/Original/train_beh.csv')
train_trd = pd.read_csv('./Data/Original/train_trd.csv')
train_tag = pd.read_csv('./Data/Original/train_tag.csv')
test_beh = pd.read_csv('./Data/Original/test_beh.csv')
test_trd = pd.read_csv('./Data/Original/test_trd.csv')
test_tag = pd.read_csv('./Data/Original/test_tag.csv')

C:\Users\jfire\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3460: DtypeWarning: Columns (7,36) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Utility functions

In [3]:
# all of the types above seem appropriate, so have little to correct there. 
#After checking types, it is useful to check the missing values
def produce_missing_report(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
    return missing_value_df

In [4]:
def reorder_columns(df, column_list):
    # Get a list of column names in the DataFrame that are NOT in the column_list
    other_columns = [col for col in df.columns if col not in column_list]

    # Concatenate the other_columns and the column_list to get the new order of columns
    new_order = other_columns + column_list

    return df[new_order]

In [5]:
def replace_with_mode(s, value):
    mode_value = s.mode()[0]
    s = s.fillna(mode_value)
    if value in s.values:
        s = s.replace({value: mode_value})

    return s

## Train_beh

In [6]:
train_beh.head()

,id,flag,page_no,page_tm
0,U64F0C9,0,SZA,6/30/2019 12:44
1,U64F0C9,0,CQE,6/25/2019 16:15
2,U64F0C9,0,AAO,6/30/2019 12:44
3,U64F0C9,0,CQE,6/17/2019 13:50
4,U64F0C9,0,AAO,6/17/2019 13:50


'flag' should be the last column so let's do that

In [7]:
train_beh = train_beh[['id', 'page_no', 'page_tm', 'flag']]
train_beh.head()

,id,page_no,page_tm,flag
0,U64F0C9,SZA,6/30/2019 12:44,0
1,U64F0C9,CQE,6/25/2019 16:15,0
2,U64F0C9,AAO,6/30/2019 12:44,0
3,U64F0C9,CQE,6/17/2019 13:50,0
4,U64F0C9,AAO,6/17/2019 13:50,0


In [8]:
train_beh.shape

(934282, 4)

In [9]:
train_beh.columns

Index(['id', 'page_no', 'page_tm', 'flag'], dtype='object')

In [10]:
train_beh.dtypes

id         object
page_no    object
page_tm    object
flag        int64
dtype: object

Let's sort the dataframe by id and date-time (page_tm)

In [11]:
train_beh_2 = reorder_columns(train_beh, ['flag']) 
train_beh_2.head()

,id,page_no,page_tm,flag
0,U64F0C9,SZA,6/30/2019 12:44,0
1,U64F0C9,CQE,6/25/2019 16:15,0
2,U64F0C9,AAO,6/30/2019 12:44,0
3,U64F0C9,CQE,6/17/2019 13:50,0
4,U64F0C9,AAO,6/17/2019 13:50,0


Now we'll rename the columns for ease of reference


In [12]:
train_beh_2.rename(columns = {'id':'Id', 'page_no':'Page Name', 'page_tm':'Visit Time', 'flag':'Flag'}, 
                                 inplace = True)
train_beh_2.head()

,Id,Page Name,Visit Time,Flag
0,U64F0C9,SZA,6/30/2019 12:44,0
1,U64F0C9,CQE,6/25/2019 16:15,0
2,U64F0C9,AAO,6/30/2019 12:44,0
3,U64F0C9,CQE,6/17/2019 13:50,0
4,U64F0C9,AAO,6/17/2019 13:50,0


In [13]:
produce_missing_report(train_beh_2)

,percent_missing
Id,0.0
Page Name,0.0
Visit Time,0.0
Flag,0.0


## Test_beh

In [14]:
test_beh.head()

,id,page_no,page_tm
0,U9749EC,CQA,6/25/2019 8:57
1,U9749EC,CQA,6/15/2019 15:55
2,U9749EC,CQA,6/25/2019 12:25
3,U9749EC,CQE,6/25/2019 12:26
4,U9749EC,CQE,6/25/2019 12:25


We are missing the flag column so we will have to create this column

In [15]:
test_beh['Flag'] = pd.Series(dtype = 'int64')
test_beh.head()

,id,page_no,page_tm,Flag
0,U9749EC,CQA,6/25/2019 8:57,NaN
1,U9749EC,CQA,6/15/2019 15:55,NaN
2,U9749EC,CQA,6/25/2019 12:25,NaN
3,U9749EC,CQE,6/25/2019 12:26,NaN
4,U9749EC,CQE,6/25/2019 12:25,NaN


In [16]:
test_beh.shape

(150478, 4)

In [17]:
test_beh.dtypes

id          object
page_no     object
page_tm     object
Flag       float64
dtype: object

In [18]:
test_beh_2 = test_beh.sort_values(by=['id','page_tm'])
test_beh_2.head()

,id,page_no,page_tm,Flag
111315,U4B9037,CQA,6/15/2019 21:39,NaN
111321,U4B9037,AAO,6/15/2019 21:39,NaN
111326,U4B9037,CQA,6/15/2019 21:39,NaN
111330,U4B9037,AAO,6/15/2019 21:39,NaN
111340,U4B9037,FTR,6/15/2019 21:39,NaN


In [19]:
produce_missing_report(test_beh_2)

,percent_missing
id,0.0
page_no,0.0
page_tm,0.0
Flag,100.0


In [20]:
test_beh.rename(columns = {'id':'Id', 'page_no':'Page Name', 'page_tm':'Visit Time'}, 
                                 inplace = True)
test_beh.head()

,Id,Page Name,Visit Time,Flag
0,U9749EC,CQA,6/25/2019 8:57,NaN
1,U9749EC,CQA,6/15/2019 15:55,NaN
2,U9749EC,CQA,6/25/2019 12:25,NaN
3,U9749EC,CQE,6/25/2019 12:26,NaN
4,U9749EC,CQE,6/25/2019 12:25,NaN


## Train_trd

In [21]:
train_trd.head()

,id,flag,Dat_Flg1_Cd,Dat_Flg3_Cd,Trx_Cod1_Cd,Trx_Cod2_Cd,trx_tm,cny_trx_amt
0,UFDC88A,0,B,B,1,116,2019-06-20 07:15:28,-127.99
1,UFDC88A,0,B,B,1,116,2019-06-16 10:09:13,-55.88
2,UFDC88A,0,B,B,1,136,2019-05-14 16:11:32,-557.00
3,UFDC88A,0,B,B,1,136,2019-05-19 21:54:40,-77.80
4,UFDC88A,0,B,B,1,113,2019-06-18 08:23:59,-271.62


In [22]:
train_trd.shape

(1367211, 8)

In [23]:
train_trd.columns

Index(['id', 'flag', 'Dat_Flg1_Cd', 'Dat_Flg3_Cd', 'Trx_Cod1_Cd',
       'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt'],
      dtype='object')

In [24]:
train_trd = reorder_columns(train_trd, ['flag'])
train_trd.head()

,id,Dat_Flg1_Cd,Dat_Flg3_Cd,Trx_Cod1_Cd,Trx_Cod2_Cd,trx_tm,cny_trx_amt,flag
0,UFDC88A,B,B,1,116,2019-06-20 07:15:28,-127.99,0
1,UFDC88A,B,B,1,116,2019-06-16 10:09:13,-55.88,0
2,UFDC88A,B,B,1,136,2019-05-14 16:11:32,-557.00,0
3,UFDC88A,B,B,1,136,2019-05-19 21:54:40,-77.80,0
4,UFDC88A,B,B,1,113,2019-06-18 08:23:59,-271.62,0


In [25]:
train_trd.dtypes

id              object
Dat_Flg1_Cd     object
Dat_Flg3_Cd     object
Trx_Cod1_Cd      int64
Trx_Cod2_Cd      int64
trx_tm          object
cny_trx_amt    float64
flag             int64
dtype: object

Let's sort the dataframe by id and date-time (page_tm)

In [26]:
train_trd_2 = train_trd.sort_values(by=['id','trx_tm'])
train_trd_2.head()

,id,Dat_Flg1_Cd,Dat_Flg3_Cd,Trx_Cod1_Cd,Trx_Cod2_Cd,trx_tm,cny_trx_amt,flag
972587,U000101,B,B,1,103,2019-05-02 10:09:18,-25.0,0
972613,U000101,B,B,1,103,2019-05-02 10:40:56,-25.0,0
972583,U000101,B,B,1,103,2019-05-02 10:41:28,-50.0,0
972612,U000101,B,B,1,103,2019-05-02 13:46:24,-1.0,0
972614,U000101,B,B,1,103,2019-05-02 18:30:53,-18.8,0


In [27]:
train_trd_2['Dat_Flg1_Cd'].value_counts()

B    1068119
C     299092
Name: Dat_Flg1_Cd, dtype: int64

In [28]:
train_trd_2['Dat_Flg3_Cd'].value_counts()

A    695630
B    661136
C     10445
Name: Dat_Flg3_Cd, dtype: int64

In [29]:
train_trd_2['Trx_Cod1_Cd'].value_counts()

1    956662
3    239078
2    171471
Name: Trx_Cod1_Cd, dtype: int64

In [30]:
train_trd_2['Trx_Cod2_Cd'].value_counts()

136    220378
132    180646
309     97218
308     77705
213     76653
111     70659
103     58876
102     55049
117     51679
134     45645
128     39153
310     37609
208     33827
209     26916
104     25921
131     24257
116     23696
123     20982
112     19309
135     19032
204     18265
101     14976
108     13932
113     10898
118     10726
133     10655
307      9858
130      9709
301      9127
201      6830
107      6560
302      4513
125      4345
207      3667
129      3338
114      3180
110      2991
109      2935
126      2417
210      2258
306      2087
205      1920
127      1893
122      1328
105       834
115       469
203       375
305       350
206       346
304       310
202       275
106       194
303       154
311       147
212       138
211         1
Name: Trx_Cod2_Cd, dtype: int64

Dat_Flg1_Cd, Dat_Flg3_Cd and Trx_Cod1_Cd are categorical variables so we would need to set up dummy variables for them but first, let's relabel the columns


In [31]:
dummies_1 = pd.get_dummies(train_trd_2['Dat_Flg1_Cd'])
dummies_2 = pd.get_dummies(train_trd_2['Dat_Flg3_Cd'])
dummies_3 = pd.get_dummies(train_trd_2['Trx_Cod1_Cd'])

Dat_Flg1_Cd and Dat_Flg3_Cd both use B and C as values so their respective dummy columns are named as such so we'll rename all them to reflect they represent

In [32]:
dummies_1.rename(columns={'B':'Withdrawal', 'C': 'Deposit'}, inplace=True)
dummies_1.head()

,Withdrawal,Deposit
972587,1,0
972613,1,0
972583,1,0
972612,1,0
972614,1,0


In [33]:
dummies_2.rename(columns={'A': 'Cash', 'B':'Cheque', 'C': 'Credit'}, inplace=True)
dummies_2.head()

,Cash,Cheque,Credit
972587,0,1,0
972613,0,1,0
972583,0,1,0
972612,0,1,0
972614,0,1,0


In [34]:
dummies_3.rename(columns={1: 'Code 1', 2:'Code 2', 3: 'Code 3'}, inplace=True)
dummies_3.head()

,Code 1,Code 2,Code 3
972587,1,0,0
972613,1,0,0
972583,1,0,0
972612,1,0,0
972614,1,0,0


Now we can merge the columns

In [35]:
train_trd_3 = pd.concat([train_trd_2, dummies_1, dummies_2, dummies_3], axis='columns')
train_trd_3.head()

,id,Dat_Flg1_Cd,Dat_Flg3_Cd,Trx_Cod1_Cd,Trx_Cod2_Cd,trx_tm,cny_trx_amt,flag,Withdrawal,Deposit,Cash,Cheque,Credit,Code 1,Code 2,Code 3
972587,U000101,B,B,1,103,2019-05-02 10:09:18,-25.0,0,1,0,0,1,0,1,0,0
972613,U000101,B,B,1,103,2019-05-02 10:40:56,-25.0,0,1,0,0,1,0,1,0,0
972583,U000101,B,B,1,103,2019-05-02 10:41:28,-50.0,0,1,0,0,1,0,1,0,0
972612,U000101,B,B,1,103,2019-05-02 13:46:24,-1.0,0,1,0,0,1,0,1,0,0
972614,U000101,B,B,1,103,2019-05-02 18:30:53,-18.8,0,1,0,0,1,0,1,0,0


Next we drop the categorical columns and move flag to the end 

In [36]:
train_trd_4 = train_trd_3.drop(['Dat_Flg1_Cd', 'Dat_Flg3_Cd', 'Trx_Cod1_Cd'], axis='columns')
train_trd_4.columns

Index(['id', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt', 'flag', 'Withdrawal',
       'Deposit', 'Cash', 'Cheque', 'Credit', 'Code 1', 'Code 2', 'Code 3'],
      dtype='object')

In [37]:
end = ['Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt', 'flag']
train_trd_5 = reorder_columns(train_trd_4, end)
train_trd_5.head()

,id,Withdrawal,Deposit,Cash,Cheque,Credit,Code 1,Code 2,Code 3,Trx_Cod2_Cd,trx_tm,cny_trx_amt,flag
972587,U000101,1,0,0,1,0,1,0,0,103,2019-05-02 10:09:18,-25.0,0
972613,U000101,1,0,0,1,0,1,0,0,103,2019-05-02 10:40:56,-25.0,0
972583,U000101,1,0,0,1,0,1,0,0,103,2019-05-02 10:41:28,-50.0,0
972612,U000101,1,0,0,1,0,1,0,0,103,2019-05-02 13:46:24,-1.0,0
972614,U000101,1,0,0,1,0,1,0,0,103,2019-05-02 18:30:53,-18.8,0


Let's rename the remaining columns

In [38]:
train_trd_5.rename(columns = {'id':'Id', 'Trx_Cod2_Cd': 'Transaction Code 2', 'trx_tm': 'Transaction time', 
                              'cny_trx_amt': 'Transaction amount', 'flag': 'Flag'}, 
                                 inplace = True)
train_trd_5.head()

,Id,Withdrawal,Deposit,Cash,Cheque,Credit,Code 1,Code 2,Code 3,Transaction Code 2,Transaction time,Transaction amount,Flag
972587,U000101,1,0,0,1,0,1,0,0,103,2019-05-02 10:09:18,-25.0,0
972613,U000101,1,0,0,1,0,1,0,0,103,2019-05-02 10:40:56,-25.0,0
972583,U000101,1,0,0,1,0,1,0,0,103,2019-05-02 10:41:28,-50.0,0
972612,U000101,1,0,0,1,0,1,0,0,103,2019-05-02 13:46:24,-1.0,0
972614,U000101,1,0,0,1,0,1,0,0,103,2019-05-02 18:30:53,-18.8,0


In [39]:
produce_missing_report(train_trd_5)

,percent_missing
Id,0.0
Withdrawal,0.0
Deposit,0.0
Cash,0.0
Cheque,0.0
Credit,0.0
Code 1,0.0
Code 2,0.0
Code 3,0.0
Transaction Code 2,0.0


## Test_trd

In [40]:
test_trd.head()

,id,Dat_Flg1_Cd,Dat_Flg3_Cd,Trx_Cod1_Cd,Trx_Cod2_Cd,trx_tm,cny_trx_amt
0,U984422,B,B,1,136,2019-06-23 09:39:27,-119.0
1,U984422,B,B,1,136,2019-06-27 13:56:11,-360.0
2,U984422,B,B,1,136,2019-06-23 09:57:00,-197.0
3,U984422,B,B,1,103,2019-06-20 18:16:38,-24.0
4,U984422,B,B,1,103,2019-06-16 07:36:50,-7.0


In [41]:
test_trd.shape

(214815, 7)

In [42]:
test_trd.columns

Index(['id', 'Dat_Flg1_Cd', 'Dat_Flg3_Cd', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd',
       'trx_tm', 'cny_trx_amt'],
      dtype='object')

In [43]:
test_trd.dtypes

id              object
Dat_Flg1_Cd     object
Dat_Flg3_Cd     object
Trx_Cod1_Cd      int64
Trx_Cod2_Cd      int64
trx_tm          object
cny_trx_amt    float64
dtype: object

In [44]:
test_trd_2 = test_trd.sort_values(by=['id','trx_tm'])
test_trd_2.head()

,id,Dat_Flg1_Cd,Dat_Flg3_Cd,Trx_Cod1_Cd,Trx_Cod2_Cd,trx_tm,cny_trx_amt
145311,U4ACAEA,B,B,1,102,2019-05-03 13:23:34,-117.0
145313,U4ACAEA,B,B,1,102,2019-05-13 18:28:56,-59.0
145317,U4ACAEA,B,B,1,102,2019-05-19 19:13:32,-78.8
145308,U4ACAEA,B,B,1,102,2019-05-22 13:05:53,-137.0
145307,U4ACAEA,C,B,3,309,2019-05-26 17:31:12,334.0


In [45]:
test_trd_2['Flag'] = pd.Series(dtype = 'int64')
test_trd_2.head()

,id,Dat_Flg1_Cd,Dat_Flg3_Cd,Trx_Cod1_Cd,Trx_Cod2_Cd,trx_tm,cny_trx_amt,Flag
145311,U4ACAEA,B,B,1,102,2019-05-03 13:23:34,-117.0,NaN
145313,U4ACAEA,B,B,1,102,2019-05-13 18:28:56,-59.0,NaN
145317,U4ACAEA,B,B,1,102,2019-05-19 19:13:32,-78.8,NaN
145308,U4ACAEA,B,B,1,102,2019-05-22 13:05:53,-137.0,NaN
145307,U4ACAEA,C,B,3,309,2019-05-26 17:31:12,334.0,NaN


In [46]:
dummies_1 = pd.get_dummies(test_trd_2['Dat_Flg1_Cd'])
dummies_2 = pd.get_dummies(test_trd_2['Dat_Flg3_Cd'])
dummies_3 = pd.get_dummies(test_trd_2['Trx_Cod1_Cd'])

In [47]:
dummies_1.rename(columns={'B':'Withdrawal', 'C': 'Deposit'}, inplace=True)
dummies_1.head()

,Withdrawal,Deposit
145311,1,0
145313,1,0
145317,1,0
145308,1,0
145307,0,1


In [48]:
dummies_1.rename(columns={'B':'Withdrawal', 'C': 'Deposit'}, inplace=True)
dummies_1.head()

,Withdrawal,Deposit
145311,1,0
145313,1,0
145317,1,0
145308,1,0
145307,0,1


In [49]:
dummies_2.rename(columns={'A': 'Cash', 'B':'Cheque', 'C': 'Credit'}, inplace=True)
dummies_2.head()

,Cash,Cheque,Credit
145311,0,1,0
145313,0,1,0
145317,0,1,0
145308,0,1,0
145307,0,1,0


In [50]:
dummies_3.rename(columns={1: 'Code 1', 2:'Code 2', 3: 'Code 3'}, inplace=True)
dummies_3.head()

,Code 1,Code 2,Code 3
145311,1,0,0
145313,1,0,0
145317,1,0,0
145308,1,0,0
145307,0,0,1


In [51]:
test_trd_3 = pd.concat([test_trd_2, dummies_1, dummies_2, dummies_3], axis='columns')
test_trd_3.head()

,id,Dat_Flg1_Cd,Dat_Flg3_Cd,Trx_Cod1_Cd,Trx_Cod2_Cd,trx_tm,cny_trx_amt,Flag,Withdrawal,Deposit,Cash,Cheque,Credit,Code 1,Code 2,Code 3
145311,U4ACAEA,B,B,1,102,2019-05-03 13:23:34,-117.0,NaN,1,0,0,1,0,1,0,0
145313,U4ACAEA,B,B,1,102,2019-05-13 18:28:56,-59.0,NaN,1,0,0,1,0,1,0,0
145317,U4ACAEA,B,B,1,102,2019-05-19 19:13:32,-78.8,NaN,1,0,0,1,0,1,0,0
145308,U4ACAEA,B,B,1,102,2019-05-22 13:05:53,-137.0,NaN,1,0,0,1,0,1,0,0
145307,U4ACAEA,C,B,3,309,2019-05-26 17:31:12,334.0,NaN,0,1,0,1,0,0,0,1


In [52]:
test_trd_4 = test_trd_3.drop(['Dat_Flg1_Cd', 'Dat_Flg3_Cd', 'Trx_Cod1_Cd'], axis='columns')
test_trd_4.columns

Index(['id', 'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt', 'Flag', 'Withdrawal',
       'Deposit', 'Cash', 'Cheque', 'Credit', 'Code 1', 'Code 2', 'Code 3'],
      dtype='object')

In [53]:
end = ['Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt', 'Flag']
test_trd_5 = reorder_columns(test_trd_4, end)
test_trd_5.head()

,id,Withdrawal,Deposit,Cash,Cheque,Credit,Code 1,Code 2,Code 3,Trx_Cod2_Cd,trx_tm,cny_trx_amt,Flag
145311,U4ACAEA,1,0,0,1,0,1,0,0,102,2019-05-03 13:23:34,-117.0,NaN
145313,U4ACAEA,1,0,0,1,0,1,0,0,102,2019-05-13 18:28:56,-59.0,NaN
145317,U4ACAEA,1,0,0,1,0,1,0,0,102,2019-05-19 19:13:32,-78.8,NaN
145308,U4ACAEA,1,0,0,1,0,1,0,0,102,2019-05-22 13:05:53,-137.0,NaN
145307,U4ACAEA,0,1,0,1,0,0,0,1,309,2019-05-26 17:31:12,334.0,NaN


In [54]:
test_trd_5.rename(columns = {'id':'Id', 'Trx_Cod2_Cd': 'Transaction Code 2', 'trx_tm': 'Transaction time', 
                              'cny_trx_amt': 'Transaction amount'}, inplace = True)
test_trd_5.head()

,Id,Withdrawal,Deposit,Cash,Cheque,Credit,Code 1,Code 2,Code 3,Transaction Code 2,Transaction time,Transaction amount,Flag
145311,U4ACAEA,1,0,0,1,0,1,0,0,102,2019-05-03 13:23:34,-117.0,NaN
145313,U4ACAEA,1,0,0,1,0,1,0,0,102,2019-05-13 18:28:56,-59.0,NaN
145317,U4ACAEA,1,0,0,1,0,1,0,0,102,2019-05-19 19:13:32,-78.8,NaN
145308,U4ACAEA,1,0,0,1,0,1,0,0,102,2019-05-22 13:05:53,-137.0,NaN
145307,U4ACAEA,0,1,0,1,0,0,0,1,309,2019-05-26 17:31:12,334.0,NaN


In [55]:
produce_missing_report(test_trd_5)

,percent_missing
Id,0.0
Withdrawal,0.0
Deposit,0.0
Cash,0.0
Cheque,0.0
Credit,0.0
Code 1,0.0
Code 2,0.0
Code 3,0.0
Transaction Code 2,0.0


## Train_tag

In [56]:
train_tag.head()

,id,flag,gdr_cd,age,mrg_situ_cd,edu_deg_cd,acdm_deg_cd,deg_cd,job_year,ic_ind,...,hld_crd_card_grd_cd,crd_card_act_ind,l1y_crd_card_csm_amt_dlm_cd,atdd_type,perm_crd_lmt_cd,cur_debit_cnt,cur_credit_cnt,cur_debit_min_opn_dt_cnt,cur_credit_min_opn_dt_cnt,cur_debit_crd_lvl
0,U001B19,0,\N,23,\N,\N,\N,\N,0,0,...,20,0,0,NaN,3,0,1,-1,325,0
1,U0091B9,0,\N,26,\N,\N,\N,\N,0,0,...,10,0,0,NaN,4,0,1,-1,1683,0
2,U00A5E0,0,\N,46,\N,\N,\N,\N,0,0,...,10,0,0,NaN,1,0,1,-1,466,0
3,U00DEC9,0,\N,30,\N,\N,\N,\N,1,0,...,20,1,2,NaN,3,0,3,-1,167,0
4,U01CB9E,0,\N,30,\N,\N,\N,\N,0,0,...,20,0,0,NaN,2,0,2,-1,1,0


In [57]:
train_tag = reorder_columns(train_tag, ['flag'])
train_tag.head()

,id,gdr_cd,age,mrg_situ_cd,edu_deg_cd,acdm_deg_cd,deg_cd,job_year,ic_ind,fr_or_sh_ind,...,crd_card_act_ind,l1y_crd_card_csm_amt_dlm_cd,atdd_type,perm_crd_lmt_cd,cur_debit_cnt,cur_credit_cnt,cur_debit_min_opn_dt_cnt,cur_credit_min_opn_dt_cnt,cur_debit_crd_lvl,flag
0,U001B19,\N,23,\N,\N,\N,\N,0,0,0,...,0,0,NaN,3,0,1,-1,325,0,0
1,U0091B9,\N,26,\N,\N,\N,\N,0,0,0,...,0,0,NaN,4,0,1,-1,1683,0,0
2,U00A5E0,\N,46,\N,\N,\N,\N,0,0,0,...,0,0,NaN,1,0,1,-1,466,0,0
3,U00DEC9,\N,30,\N,\N,\N,\N,1,0,0,...,1,2,NaN,3,0,3,-1,167,0,0
4,U01CB9E,\N,30,\N,\N,\N,\N,0,0,0,...,0,0,NaN,2,0,2,-1,1,0,0


In [58]:
produce_missing_report(train_tag)

,percent_missing
id,0.000000
gdr_cd,0.000000
age,0.000000
mrg_situ_cd,0.000000
edu_deg_cd,31.149964
acdm_deg_cd,0.002505
deg_cd,52.508579
job_year,0.000000
ic_ind,0.000000
fr_or_sh_ind,0.000000


In [59]:
train_tag.dtypes

id                                 object
gdr_cd                             object
age                                 int64
mrg_situ_cd                        object
edu_deg_cd                         object
acdm_deg_cd                        object
deg_cd                             object
job_year                           object
ic_ind                             object
fr_or_sh_ind                       object
dnl_mbl_bnk_ind                    object
dnl_bind_cmb_lif_ind               object
hav_car_grp_ind                    object
hav_hou_grp_ind                    object
l6mon_agn_ind                      object
frs_agn_dt_cnt                     object
vld_rsk_ases_ind                   object
fin_rsk_ases_grd_cd                object
confirm_rsk_ases_lvl_typ_cd        object
cust_inv_rsk_endu_lvl_cd           object
l6mon_daim_aum_cd                   int64
tot_ast_lvl_cd                     object
pot_ast_lvl_cd                     object
bk1_cur_year_mon_avg_agn_amt_cd   

In [60]:
train_tag.dtypes

id                                 object
gdr_cd                             object
age                                 int64
mrg_situ_cd                        object
edu_deg_cd                         object
acdm_deg_cd                        object
deg_cd                             object
job_year                           object
ic_ind                             object
fr_or_sh_ind                       object
dnl_mbl_bnk_ind                    object
dnl_bind_cmb_lif_ind               object
hav_car_grp_ind                    object
hav_hou_grp_ind                    object
l6mon_agn_ind                      object
frs_agn_dt_cnt                     object
vld_rsk_ases_ind                   object
fin_rsk_ases_grd_cd                object
confirm_rsk_ases_lvl_typ_cd        object
cust_inv_rsk_endu_lvl_cd           object
l6mon_daim_aum_cd                   int64
tot_ast_lvl_cd                     object
pot_ast_lvl_cd                     object
bk1_cur_year_mon_avg_agn_amt_cd   

In [61]:
train_tag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39923 entries, 0 to 39922
Data columns (total 43 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   id                               39923 non-null  object
 1   gdr_cd                           39923 non-null  object
 2   age                              39923 non-null  int64 
 3   mrg_situ_cd                      39923 non-null  object
 4   edu_deg_cd                       27487 non-null  object
 5   acdm_deg_cd                      39922 non-null  object
 6   deg_cd                           18960 non-null  object
 7   job_year                         39923 non-null  object
 8   ic_ind                           39923 non-null  object
 9   fr_or_sh_ind                     39923 non-null  object
 10  dnl_mbl_bnk_ind                  39923 non-null  object
 11  dnl_bind_cmb_lif_ind             39923 non-null  object
 12  hav_car_grp_ind                 

In [62]:
train_tag['gdr_cd'].value_counts()

M     23015
F     16172
\N      736
Name: gdr_cd, dtype: int64

In [63]:
#Replace '\\N' with the modal class, i.e. M
train_tag['gdr_cd'].replace(to_replace={'\\N': 'M'}, inplace = True)
train_tag['gdr_cd'].value_counts()

M    23751
F    16172
Name: gdr_cd, dtype: int64

In [64]:
train_tag['mrg_situ_cd'].value_counts()

A     19119
B     18598
O      1035
\N      736
~       248
Z       187
Name: mrg_situ_cd, dtype: int64

In [65]:
#Replace '\\N' with the modal class, i.e. A
train_tag['mrg_situ_cd'].replace(to_replace={'A':'Married', 'B':'Never Married', 'O':'Separated',
                                                          '\\N':'Married', '~':'Divorced', 'Z':'Widowed'},
                                 inplace = True)
train_tag['mrg_situ_cd'].value_counts()

Married          19855
Never Married    18598
Separated         1035
Divorced           248
Widowed            187
Name: mrg_situ_cd, dtype: int64

In [66]:
train_tag['edu_deg_cd'].value_counts()

F     6917
C     6695
B     6672
K     2312
Z     2097
G      953
A      889
\N     736
~      108
M       54
L       33
D       20
J        1
Name: edu_deg_cd, dtype: int64

In [67]:
#Replace '\\N' with the modal class, i.e. 0
train_tag['edu_deg_cd'].replace(to_replace={'F':'Bachelors', 'C':'Upper Secondary', 'B':'Post-secondary', 'Z':'Masters',
                                            'K':'Upper Secondary', 'G':'Bachelors', 'A':'Bachelors',  '\\N':'Doctorate', 
                                           '~':'Doctorate', 'M':'Post-secondary', 'L':'Post-secondary', 'D':'Masters',
                                            'J':'Masters'}, inplace = True)
train_tag['edu_deg_cd'].value_counts()

Upper Secondary    9007
Bachelors          8759
Post-secondary     6759
Masters            2118
Doctorate           844
Name: edu_deg_cd, dtype: int64

In [68]:
train_tag['loan_act_ind'].value_counts()

0     37224
1      2206
\N      493
Name: loan_act_ind, dtype: int64

In [69]:
#Replace '\\N' with the modal class, i.e. 0
train_tag['loan_act_ind'].replace(to_replace={'\\N':'0'}, inplace = True)
train_tag['loan_act_ind'].value_counts()

0    37717
1     2206
Name: loan_act_ind, dtype: int64

In [70]:
drop_columns = ['id', 'atdd_type', 'acdm_deg_cd', 'deg_cd', 'dnl_bind_cmb_lif_ind', 'perm_crd_lmt_cd', 'hld_crd_card_grd_cd',
                'l6mon_daim_aum_cd', 'cust_inv_rsk_endu_lvl_cd', 'confirm_rsk_ases_lvl_typ_cd', 'fin_rsk_ases_grd_cd', 
                'bk1_cur_year_mon_avg_agn_amt_cd']

train_tag = train_tag.drop(drop_columns, axis=1)
train_tag.head(10)

,gdr_cd,age,mrg_situ_cd,edu_deg_cd,job_year,ic_ind,fr_or_sh_ind,dnl_mbl_bnk_ind,hav_car_grp_ind,hav_hou_grp_ind,...,ovd_30d_loan_tot_cnt,his_lng_ovd_day,crd_card_act_ind,l1y_crd_card_csm_amt_dlm_cd,cur_debit_cnt,cur_credit_cnt,cur_debit_min_opn_dt_cnt,cur_credit_min_opn_dt_cnt,cur_debit_crd_lvl,flag
0,M,23,Married,Doctorate,0,0,0,0,0,0,...,0,0,0,0,0,1,-1,325,0,0
1,M,26,Married,Doctorate,0,0,0,1,0,0,...,0,0,0,0,0,1,-1,1683,0,0
2,M,46,Married,Doctorate,0,0,0,0,0,0,...,0,0,0,0,0,1,-1,466,0,0
3,M,30,Married,Doctorate,1,0,0,0,1,0,...,0,0,1,2,0,3,-1,167,0,0
4,M,30,Married,Doctorate,0,0,0,0,0,0,...,0,0,0,0,0,2,-1,1,0,0
5,M,45,Married,Doctorate,3,0,1,0,1,0,...,0,0,1,5,0,2,-1,1323,0,0
6,M,38,Married,Doctorate,0,0,0,0,1,0,...,0,0,1,5,0,1,-1,1872,0,0
7,M,34,Married,Doctorate,0,0,0,1,0,0,...,0,0,1,3,0,1,-1,867,0,0
8,M,30,Married,Doctorate,0,0,0,0,0,0,...,0,0,1,2,0,3,-1,256,0,0
9,M,30,Married,Doctorate,1,0,0,0,0,0,...,0,0,1,1,0,2,-1,319,0,0


In [71]:
hold_columns = ['job_year', 'edu_deg_cd', 'ic_ind', 'fr_or_sh_ind', 'dnl_mbl_bnk_ind', 'hav_car_grp_ind', 'hav_hou_grp_ind', 
                'l6mon_agn_ind', 'frs_agn_dt_cnt', 'vld_rsk_ases_ind', 'tot_ast_lvl_cd', 'pot_ast_lvl_cd', 
                'l12mon_buy_fin_mng_whl_tms', 'l12_mon_fnd_buy_whl_tms', 'l12_mon_insu_buy_whl_tms', 'l12_mon_gld_buy_whl_tms',
                'loan_act_ind', 'pl_crd_lmt_cd', 'ovd_30d_loan_tot_cnt', 'his_lng_ovd_day', 'crd_card_act_ind', 
                'l1y_crd_card_csm_amt_dlm_cd', 'cur_debit_cnt', 'cur_credit_cnt',  'cur_debit_min_opn_dt_cnt', 
                'cur_credit_min_opn_dt_cnt', 'cur_debit_crd_lvl']

for column in hold_columns: 
    train_tag[column] = replace_with_mode(train_tag[column], '\\N')
    
train_tag    

C:\Users\jfire\AppData\Local\Temp\ipykernel_13500\1920221554.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if value in s.values:


,gdr_cd,age,mrg_situ_cd,edu_deg_cd,job_year,ic_ind,fr_or_sh_ind,dnl_mbl_bnk_ind,hav_car_grp_ind,hav_hou_grp_ind,...,ovd_30d_loan_tot_cnt,his_lng_ovd_day,crd_card_act_ind,l1y_crd_card_csm_amt_dlm_cd,cur_debit_cnt,cur_credit_cnt,cur_debit_min_opn_dt_cnt,cur_credit_min_opn_dt_cnt,cur_debit_crd_lvl,flag
0,M,23,Married,Doctorate,0,0,0,0,0,0,...,0,0,0,0,0,1,-1,325,0,0
1,M,26,Married,Doctorate,0,0,0,1,0,0,...,0,0,0,0,0,1,-1,1683,0,0
2,M,46,Married,Doctorate,0,0,0,0,0,0,...,0,0,0,0,0,1,-1,466,0,0
3,M,30,Married,Doctorate,1,0,0,0,1,0,...,0,0,1,2,0,3,-1,167,0,0
4,M,30,Married,Doctorate,0,0,0,0,0,0,...,0,0,0,0,0,2,-1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39918,M,41,Never Married,Upper Secondary,0,0,1,1,1,0,...,0,0,1,5,1,2,1833,1823,10,0
39919,F,47,Never Married,Upper Secondary,0,0,1,0,0,0,...,0,0,1,3,0,2,-1,752,0,0
39920,M,44,Never Married,Upper Secondary,16,0,0,0,0,0,...,0,0,1,1,0,1,-1,958,0,0
39921,M,29,Married,Upper Secondary,0,0,0,0,1,0,...,0,0,1,5,0,1,-1,1138,0,0


In [72]:
train_tag.columns

Index(['gdr_cd', 'age', 'mrg_situ_cd', 'edu_deg_cd', 'job_year', 'ic_ind',
       'fr_or_sh_ind', 'dnl_mbl_bnk_ind', 'hav_car_grp_ind', 'hav_hou_grp_ind',
       'l6mon_agn_ind', 'frs_agn_dt_cnt', 'vld_rsk_ases_ind', 'tot_ast_lvl_cd',
       'pot_ast_lvl_cd', 'l12mon_buy_fin_mng_whl_tms',
       'l12_mon_fnd_buy_whl_tms', 'l12_mon_insu_buy_whl_tms',
       'l12_mon_gld_buy_whl_tms', 'loan_act_ind', 'pl_crd_lmt_cd',
       'ovd_30d_loan_tot_cnt', 'his_lng_ovd_day', 'crd_card_act_ind',
       'l1y_crd_card_csm_amt_dlm_cd', 'cur_debit_cnt', 'cur_credit_cnt',
       'cur_debit_min_opn_dt_cnt', 'cur_credit_min_opn_dt_cnt',
       'cur_debit_crd_lvl', 'flag'],
      dtype='object')

In [73]:
old_col = ['gdr_cd', 'age', 'mrg_situ_cd', 'edu_deg_cd', 'job_year', 'ic_ind', 'fr_or_sh_ind', 'dnl_mbl_bnk_ind',
           'hav_car_grp_ind', 'hav_hou_grp_ind', 'l6mon_agn_ind', 'frs_agn_dt_cnt', 'vld_rsk_ases_ind', 'tot_ast_lvl_cd',
           'pot_ast_lvl_cd', 'l12mon_buy_fin_mng_whl_tms', 'l12_mon_fnd_buy_whl_tms', 'l12_mon_insu_buy_whl_tms', 
           'l12_mon_gld_buy_whl_tms', 'loan_act_ind', 'pl_crd_lmt_cd', 'ovd_30d_loan_tot_cnt', 'his_lng_ovd_day', 
           'crd_card_act_ind', 'l1y_crd_card_csm_amt_dlm_cd', 'cur_debit_cnt', 'cur_credit_cnt', 'cur_debit_min_opn_dt_cnt', 
           'cur_credit_min_opn_dt_cnt', 'cur_debit_crd_lvl', 'flag']

new_col = ['Gender', 'Age', 'Marital Status', 'Education', 'Years Worked', 'Commercial', 'Shareholder', 'Has App', 'Owns Car', 
           'Owns House','Wages Paid', 'Last Payment', 'Investment Risk', 'Total Asset Code', 'Potential Asset Code', 
           'Financial Producs', 'Fund Purchases', 'Insurance Purchases', 'Gold Purchases', 'Has Loan', 'Total Loans', 
           'Overdue Loans', 'Overdue Days', 'Has Credit Cards', 'Credit Use', 'Debit Cards', 'Credit Cards', 'Debit ', 
           'Credit', 'Debit Card Level', 'Flag']

print(len(old_col), len(new_col))

31 31


In [74]:
col_dict = {old_col: new_col for old_col, new_col in zip(old_col, new_col)}

train_tag.rename(columns = col_dict, inplace = True)
train_tag.head()

,Gender,Age,Marital Status,Education,Years Worked,Commercial,Shareholder,Has App,Owns Car,Owns House,...,Overdue Loans,Overdue Days,Has Credit Cards,Credit Use,Debit Cards,Credit Cards,Debit,Credit,Debit Card Level,Flag
0,M,23,Married,Doctorate,0,0,0,0,0,0,...,0,0,0,0,0,1,-1,325,0,0
1,M,26,Married,Doctorate,0,0,0,1,0,0,...,0,0,0,0,0,1,-1,1683,0,0
2,M,46,Married,Doctorate,0,0,0,0,0,0,...,0,0,0,0,0,1,-1,466,0,0
3,M,30,Married,Doctorate,1,0,0,0,1,0,...,0,0,1,2,0,3,-1,167,0,0
4,M,30,Married,Doctorate,0,0,0,0,0,0,...,0,0,0,0,0,2,-1,1,0,0


In [75]:
train_tag.to_csv('./Data/cleaned_train_tag.csv')

## Test_tag

In [76]:
test_tag.head()

,id,gdr_cd,age,mrg_situ_cd,edu_deg_cd,acdm_deg_cd,deg_cd,job_year,ic_ind,fr_or_sh_ind,...,hld_crd_card_grd_cd,crd_card_act_ind,l1y_crd_card_csm_amt_dlm_cd,atdd_type,perm_crd_lmt_cd,cur_debit_cnt,cur_credit_cnt,cur_debit_min_opn_dt_cnt,cur_credit_min_opn_dt_cnt,cur_debit_crd_lvl
0,UC37930,F,39,B,C,31,~,0,0,0,...,20,1,4,NaN,6,3,4,4378,683,20
1,U5BE130,F,40,B,Z,Z,NaN,1,0,0,...,10,0,1,NaN,2,1,1,1078,1637,10
2,UD025AE,F,48,B,C,31,A,0,0,1,...,30,1,5,0.0,7,5,3,6361,4809,20
3,UC2D00D,M,47,B,A,Z,C,0,0,1,...,10,1,5,0.0,7,8,3,8032,5447,40
4,UAF705D,F,36,B,B,30,~,2,0,0,...,20,1,5,NaN,8,4,2,5885,1748,10


In [77]:
produce_missing_report(test_tag)

,percent_missing
id,0.000000
gdr_cd,0.000000
age,0.000000
mrg_situ_cd,0.000000
edu_deg_cd,30.866667
acdm_deg_cd,0.000000
deg_cd,52.216667
job_year,0.000000
ic_ind,0.000000
fr_or_sh_ind,0.000000


In [78]:
test_tag.dtypes

id                                  object
gdr_cd                              object
age                                  int64
mrg_situ_cd                         object
edu_deg_cd                          object
acdm_deg_cd                         object
deg_cd                              object
job_year                            object
ic_ind                              object
fr_or_sh_ind                        object
dnl_mbl_bnk_ind                     object
dnl_bind_cmb_lif_ind                object
hav_car_grp_ind                     object
hav_hou_grp_ind                     object
l6mon_agn_ind                       object
frs_agn_dt_cnt                      object
vld_rsk_ases_ind                    object
fin_rsk_ases_grd_cd                 object
confirm_rsk_ases_lvl_typ_cd         object
cust_inv_rsk_endu_lvl_cd            object
l6mon_daim_aum_cd                    int64
tot_ast_lvl_cd                      object
pot_ast_lvl_cd                      object
bk1_cur_yea

In [79]:
test_tag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 42 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               6000 non-null   object 
 1   gdr_cd                           6000 non-null   object 
 2   age                              6000 non-null   int64  
 3   mrg_situ_cd                      6000 non-null   object 
 4   edu_deg_cd                       4148 non-null   object 
 5   acdm_deg_cd                      6000 non-null   object 
 6   deg_cd                           2867 non-null   object 
 7   job_year                         6000 non-null   object 
 8   ic_ind                           6000 non-null   object 
 9   fr_or_sh_ind                     6000 non-null   object 
 10  dnl_mbl_bnk_ind                  6000 non-null   object 
 11  dnl_bind_cmb_lif_ind             6000 non-null   object 
 12  hav_car_grp_ind     

In [80]:
test_tag['gdr_cd'].value_counts()

M     3420
F     2452
\N     128
Name: gdr_cd, dtype: int64

In [81]:
#Replace '\\N' with the modal class, i.e. M
test_tag['gdr_cd'].replace(to_replace={'\\N': 'M'}, inplace = True)
test_tag['gdr_cd'].value_counts()

M    3548
F    2452
Name: gdr_cd, dtype: int64

In [82]:
test_tag['mrg_situ_cd'].value_counts()

A     2902
B     2756
O      147
\N     128
~       41
Z       26
Name: mrg_situ_cd, dtype: int64

In [83]:
#Replace '\\N' with the modal class, i.e. A
test_tag['mrg_situ_cd'].replace(to_replace={'A':'Married', 'B':'Never Married', 'O':'Separated',
                                                          '\\N':'Married', '~':'Divorced', 'Z':'Widowed'},
                                 inplace = True)
test_tag['mrg_situ_cd'].value_counts()

Married          3030
Never Married    2756
Separated         147
Divorced           41
Widowed            26
Name: mrg_situ_cd, dtype: int64

In [84]:
test_tag['edu_deg_cd'].value_counts()

B     1027
C     1019
F     1005
K      329
Z      322
G      146
A      141
\N     128
~       21
M        8
D        1
L        1
Name: edu_deg_cd, dtype: int64

In [85]:
#Replace '\\N' with the modal class, i.e. 0
test_tag['edu_deg_cd'].replace(to_replace={'F':'Bachelors', 'C':'Upper Secondary', 'B':'Post-secondary', 'Z':'Masters',
                                            'K':'Upper Secondary', 'G':'Bachelors', 'A':'Bachelors',  '\\N':'Doctorate', 
                                           '~':'Doctorate', 'M':'Post-secondary', 'L':'Post-secondary', 'D':'Masters',
                                            'J':'Masters'}, inplace = True)
test_tag['edu_deg_cd'].value_counts()

Upper Secondary    1348
Bachelors          1292
Post-secondary     1036
Masters             323
Doctorate           149
Name: edu_deg_cd, dtype: int64

In [86]:
test_tag['loan_act_ind'].value_counts()

0     5585
1      335
\N      80
Name: loan_act_ind, dtype: int64

In [87]:
#Replace '\\N' with the modal class, i.e. 0
test_tag['loan_act_ind'].replace(to_replace={'\\N':'0'}, inplace = True)
test_tag['loan_act_ind'].value_counts()

0    5665
1     335
Name: loan_act_ind, dtype: int64

In [88]:
drop_columns = ['id', 'atdd_type', 'acdm_deg_cd', 'deg_cd', 'dnl_bind_cmb_lif_ind', 'perm_crd_lmt_cd', 'hld_crd_card_grd_cd',
                'l6mon_daim_aum_cd', 'cust_inv_rsk_endu_lvl_cd', 'confirm_rsk_ases_lvl_typ_cd', 'fin_rsk_ases_grd_cd', 
                'bk1_cur_year_mon_avg_agn_amt_cd']

test_tag = test_tag.drop(drop_columns, axis=1)
test_tag.head(10)

,gdr_cd,age,mrg_situ_cd,edu_deg_cd,job_year,ic_ind,fr_or_sh_ind,dnl_mbl_bnk_ind,hav_car_grp_ind,hav_hou_grp_ind,...,pl_crd_lmt_cd,ovd_30d_loan_tot_cnt,his_lng_ovd_day,crd_card_act_ind,l1y_crd_card_csm_amt_dlm_cd,cur_debit_cnt,cur_credit_cnt,cur_debit_min_opn_dt_cnt,cur_credit_min_opn_dt_cnt,cur_debit_crd_lvl
0,F,39,Never Married,Upper Secondary,0,0,0,1,1,0,...,2,0,0,1,4,3,4,4378,683,20
1,F,40,Never Married,Masters,1,0,0,1,0,0,...,0,0,0,0,1,1,1,1078,1637,10
2,F,48,Never Married,Upper Secondary,0,0,1,1,1,1,...,0,0,0,1,5,5,3,6361,4809,20
3,M,47,Never Married,Bachelors,0,0,1,1,1,0,...,0,0,0,1,5,8,3,8032,5447,40
4,F,36,Never Married,Post-secondary,2,0,0,1,1,0,...,0,0,0,1,5,4,2,5885,1748,10
5,M,36,Never Married,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,1,-1,766,0
6,F,24,Married,NaN,0,0,0,0,1,0,...,0,0,0,1,3,0,2,-1,549,0
7,F,24,Married,NaN,1,0,0,0,0,0,...,0,0,0,0,1,0,2,-1,539,0
8,M,27,Married,NaN,3,0,0,1,1,0,...,0,0,0,0,1,0,3,-1,513,0
9,F,31,Never Married,NaN,2,0,0,0,0,0,...,0,0,0,0,0,0,2,-1,463,0


In [89]:
hold_columns = ['job_year', 'edu_deg_cd', 'ic_ind', 'fr_or_sh_ind', 'dnl_mbl_bnk_ind', 'hav_car_grp_ind', 'hav_hou_grp_ind', 
                'l6mon_agn_ind', 'frs_agn_dt_cnt', 'vld_rsk_ases_ind', 'tot_ast_lvl_cd', 'pot_ast_lvl_cd', 
                'l12mon_buy_fin_mng_whl_tms', 'l12_mon_fnd_buy_whl_tms', 'l12_mon_insu_buy_whl_tms', 'l12_mon_gld_buy_whl_tms',
                'loan_act_ind', 'pl_crd_lmt_cd', 'ovd_30d_loan_tot_cnt', 'his_lng_ovd_day', 'crd_card_act_ind', 
                'l1y_crd_card_csm_amt_dlm_cd', 'cur_debit_cnt', 'cur_credit_cnt',  'cur_debit_min_opn_dt_cnt', 
                'cur_credit_min_opn_dt_cnt', 'cur_debit_crd_lvl']

for column in hold_columns: 
    test_tag[column] = replace_with_mode(test_tag[column], '\\N')
    
test_tag.head()

C:\Users\jfire\AppData\Local\Temp\ipykernel_13500\1920221554.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if value in s.values:


,gdr_cd,age,mrg_situ_cd,edu_deg_cd,job_year,ic_ind,fr_or_sh_ind,dnl_mbl_bnk_ind,hav_car_grp_ind,hav_hou_grp_ind,...,pl_crd_lmt_cd,ovd_30d_loan_tot_cnt,his_lng_ovd_day,crd_card_act_ind,l1y_crd_card_csm_amt_dlm_cd,cur_debit_cnt,cur_credit_cnt,cur_debit_min_opn_dt_cnt,cur_credit_min_opn_dt_cnt,cur_debit_crd_lvl
0,F,39,Never Married,Upper Secondary,0,0,0,1,1,0,...,2,0,0,1,4,3,4,4378,683,20
1,F,40,Never Married,Masters,1,0,0,1,0,0,...,0,0,0,0,1,1,1,1078,1637,10
2,F,48,Never Married,Upper Secondary,0,0,1,1,1,1,...,0,0,0,1,5,5,3,6361,4809,20
3,M,47,Never Married,Bachelors,0,0,1,1,1,0,...,0,0,0,1,5,8,3,8032,5447,40
4,F,36,Never Married,Post-secondary,2,0,0,1,1,0,...,0,0,0,1,5,4,2,5885,1748,10


In [90]:
test_tag.columns

Index(['gdr_cd', 'age', 'mrg_situ_cd', 'edu_deg_cd', 'job_year', 'ic_ind',
       'fr_or_sh_ind', 'dnl_mbl_bnk_ind', 'hav_car_grp_ind', 'hav_hou_grp_ind',
       'l6mon_agn_ind', 'frs_agn_dt_cnt', 'vld_rsk_ases_ind', 'tot_ast_lvl_cd',
       'pot_ast_lvl_cd', 'l12mon_buy_fin_mng_whl_tms',
       'l12_mon_fnd_buy_whl_tms', 'l12_mon_insu_buy_whl_tms',
       'l12_mon_gld_buy_whl_tms', 'loan_act_ind', 'pl_crd_lmt_cd',
       'ovd_30d_loan_tot_cnt', 'his_lng_ovd_day', 'crd_card_act_ind',
       'l1y_crd_card_csm_amt_dlm_cd', 'cur_debit_cnt', 'cur_credit_cnt',
       'cur_debit_min_opn_dt_cnt', 'cur_credit_min_opn_dt_cnt',
       'cur_debit_crd_lvl'],
      dtype='object')

In [91]:
old_col = ['gdr_cd', 'age', 'mrg_situ_cd', 'edu_deg_cd', 'job_year', 'ic_ind', 'fr_or_sh_ind', 'dnl_mbl_bnk_ind',
           'hav_car_grp_ind', 'hav_hou_grp_ind', 'l6mon_agn_ind', 'frs_agn_dt_cnt', 'vld_rsk_ases_ind', 'tot_ast_lvl_cd',
           'pot_ast_lvl_cd', 'l12mon_buy_fin_mng_whl_tms', 'l12_mon_fnd_buy_whl_tms', 'l12_mon_insu_buy_whl_tms', 
           'l12_mon_gld_buy_whl_tms', 'loan_act_ind', 'pl_crd_lmt_cd', 'ovd_30d_loan_tot_cnt', 'his_lng_ovd_day', 
           'crd_card_act_ind', 'l1y_crd_card_csm_amt_dlm_cd', 'cur_debit_cnt', 'cur_credit_cnt', 'cur_debit_min_opn_dt_cnt', 
           'cur_credit_min_opn_dt_cnt', 'cur_debit_crd_lvl']

new_col = ['Gender', 'Age', 'Marital Status', 'Education', 'Years Worked', 'Commercial', 'Shareholder', 'Has App', 'Owns Car', 
           'Owns House','Wages Paid', 'Last Payment', 'Investment Risk', 'Total Asset Code', 'Potential Asset Code', 
           'Financial Producs', 'Fund Purchases', 'Insurance Purchases', 'Gold Purchases', 'Has Loan', 'Total Loans', 
           'Overdue Loans', 'Overdue Days', 'Has Credit Cards', 'Credit Use', 'Debit Cards', 'Credit Cards', 'Debit ', 
           'Credit', 'Debit Card Level']

print(len(old_col), len(new_col))

30 30


In [92]:
col_dict = {old_col: new_col for old_col, new_col in zip(old_col, new_col)}

test_tag.rename(columns = col_dict, inplace = True)
test_tag.head()

,Gender,Age,Marital Status,Education,Years Worked,Commercial,Shareholder,Has App,Owns Car,Owns House,...,Total Loans,Overdue Loans,Overdue Days,Has Credit Cards,Credit Use,Debit Cards,Credit Cards,Debit,Credit,Debit Card Level
0,F,39,Never Married,Upper Secondary,0,0,0,1,1,0,...,2,0,0,1,4,3,4,4378,683,20
1,F,40,Never Married,Masters,1,0,0,1,0,0,...,0,0,0,0,1,1,1,1078,1637,10
2,F,48,Never Married,Upper Secondary,0,0,1,1,1,1,...,0,0,0,1,5,5,3,6361,4809,20
3,M,47,Never Married,Bachelors,0,0,1,1,1,0,...,0,0,0,1,5,8,3,8032,5447,40
4,F,36,Never Married,Post-secondary,2,0,0,1,1,0,...,0,0,0,1,5,4,2,5885,1748,10


In [93]:
produce_missing_report(test_tag)

,percent_missing
Gender,0.0
Age,0.0
Marital Status,0.0
Education,0.0
Years Worked,0.0
Commercial,0.0
Shareholder,0.0
Has App,0.0
Owns Car,0.0
Owns House,0.0


In [94]:
test_tag.to_csv('./Data/cleaned_test_tag.csv')